In [3]:
import sys
print(sys.executable)
print(sys.version)

D:\Sean\Documents\Personal\Budget\Py_budget\.venv\Scripts\python.exe
3.14.0 (tags/v3.14.0:ebf955d, Oct  7 2025, 10:15:03) [MSC v.1944 64 bit (AMD64)]


In [4]:
1+1

2

In [5]:
import numpy

numpy.__version__

'2.3.4'

In [6]:
from datetime import datetime
from datetime import datetime as dt  #to manipulate dates

import numpy as np  #to cleanse data
import openpyxl
import pandas as pd
import plotly.express as px  #to create interactive charts
import plotly.graph_objects as go  #to create interactive charts

In [7]:
dtype_dict = {
    'Description': 'string',
    'Amount': 'float',
    'Category': 'string',
    'Short_Desc': 'string'
}
ytd_data = pd.read_csv(
    'D:\\Sean\\Documents\\Personal\\Budget\\YTD_downloads\\2024\\Discover-2024-YearEndSummary.csv',
    dtype=dtype_dict,
    parse_dates=['Trans. Date', 'Post Date']
)
ytd_data['Sort_Date'] = pd.to_datetime(ytd_data['Trans. Date'], format='%m/%d/%Y')
ytd_data['Date'] = pd.to_datetime(ytd_data['Sort_Date'].dt.strftime('%m/%Y'), format='%m/%Y')
ytd_data['Account'] = 'discover'
ytd_data.reset_index(drop=False, inplace=True)
ytd_data.dtypes

index                   int64
Trans. Date    datetime64[ns]
Post Date      datetime64[ns]
Description    string[python]
Amount                float64
Category       string[python]
Sort_Date      datetime64[ns]
Date           datetime64[ns]
Account                object
dtype: object

In [8]:
ytd_data.tail(10)

,index,Trans. Date,Post Date,Description,Amount,Category,Sort_Date,Date,Account
917,917,2024-12-26,2024-12-26,AMAZON MKTPL*ZE7598I02 AMZN.COM/BILLWA2JL8K0W13WY,14.24,Merchandise,2024-12-26,2024-12-01,discover
918,918,2024-12-27,2024-12-27,ADOBE *ADOBE 408-536-6000 CAITEM TRANSFERRED F...,59.99,Merchandise,2024-12-27,2024-12-01,discover
919,919,2024-12-27,2024-12-27,AMAZON.COM*ZE16I8V30 AMZN.COM/BILLWA47WWZB54K4R,43.84,Merchandise,2024-12-27,2024-12-01,discover
920,920,2024-12-27,2024-12-27,ONLYFANS 8886880458 FLITEM TRANSFERRED FROM PR...,11.90,Merchandise,2024-12-27,2024-12-01,discover
921,921,2024-12-27,2024-12-27,ONLYFANS.COM*A 8886880458 FLITEM TRANSFERRED F...,8.50,Merchandise,2024-12-27,2024-12-01,discover
922,922,2024-12-27,2024-12-27,USPS PO 0510350110 BURLINGAME CA,44.65,Government Services,2024-12-27,2024-12-01,discover
923,923,2024-12-28,2024-12-28,FASTRAK CSC 415-486-8655 CAITEM TRANSFERRED FR...,50.00,Merchandise,2024-12-28,2024-12-01,discover
924,924,2024-12-28,2024-12-28,ONLYFANS.COM*A 8886880458 FLITEM TRANSFERRED F...,5.00,Merchandise,2024-12-28,2024-12-01,discover
925,925,2024-12-29,2024-12-29,SAFEWAY #1547 BURLINGAME CA,63.28,Supermarkets,2024-12-29,2024-12-01,discover
926,926,2024-12-30,2024-12-30,GOOGLE *BUMBLE 855-836-3987 CAITEM TRANSFERRED...,15.89,Merchandise,2024-12-30,2024-12-01,discover


In [9]:
# Discover Transactions
mapping = {
    # Bills for Physical Property
    'payment': ['payment', 'RETURNED INTERNET PMT'],
    'interest': ['interest'],
    'cashback_bonus': ['cashback'],
    'fees': ['RETURNED CHECK CHARGE', 'UNRECOGNIZED TRAN CHRGBKAN', r'CSP\*CONVENIENCE FEE'],
    'fine': ['SANMATEOCOURT FINE PAY', r'TICKETTOASTER\.COM'],

    # Helping others
    'tahjei': ['the UPS store'],
    'jayelin': ['CYANFACE'],
    'don': ['justina'],
    'dad': ['gift', r'AF\*REAL'],
    'mom': ['ALL STAR PLUMBING', 'SP RING USA 8006561918 CA'],

    # Bills for Physical Property
    'cell_phone': ['tmobile', 'APPLE STORE','GOOGLE STORE MTV MOUNTAIN VIEWCA'],
    'state_farm': ['state farm'],
    'aps': ['Arizonapub'],
    'gym': ['planet fitness', 'life time', 'LA FITNESS', 'ATG USA LLC'],
    'car': ['COBBLESTONE AUTO SPA 51 PHOENIX AZ', 'AT THE SHOP SAN MATEO CA', 'CLEAN FREAK - THOMAS PHOENIX AZ',
            'TAKE 5 #667 PHOENIX AZ', 'autozone', 'dmv', 'NISSAN OF BURLINGAME BURLINGAME CA', 'OIL CHANGER',
            "WWW.DIGISTORE24.COM - 8003567947 FL", "auto parts", 'JIFFY LUBE', 'ELECTRIFY AMERICA', 'BLINK CHARGING',
            'EVGO', 'EZPASSVA', r'quickquack\*'],
    'dump': ['blue line'],

    # Bills for Essentials
    'groceries': ['safeway', 'whole', 'market', 'giant', 'frys', 'albertsons', 'Food City', 'Ralphs',
                  'NATURAL GROCERS PI PHOENIX AZ', 'INSTACART', "DEAN'S PRODUCE", "YOLO FRUIT STAND", "TRADER JOE",
                  "Sprouts Farmers", "harristeeter", "99 ranch", 'LUCKY #748', 'SHIPT',
                  'A.D.P DEPALMA FARM SAN FRANCISCOCA0001152921513049170106',
                  'THAO FAMILY FARM SAN FRANCISCOCA0001152921513049149557', 'BEARD PAPA\'S SAN MATEO CA',
                  'NICK CIPPONERI FAR', 'DRAEGER\'S', 'MARTINS', 'GROCERY OUTLET', 'SAVE-A-LOT'],
    'amazon': ['amzn'],
    'barbershop': ['barb', "COASTAL CURL"],
    'taxes': ['FREETAXUSA.COM 877-2699027 UT'],
    'vpn_router': ['FLASHROUTERS'],
    'conferences': ['MEN HAVING BABIES', 'DEANS PRODUCE'],

    # Bills for Discretionary Food Spending
    'fast_food': ['dunkin', 'taco', 'mcdonald', 'Jimmy', 'cafe rio', 'glow tea', 'pizza', 'pollo', 'nora asian',
                  'wow wow', 'burger king', 'panda',
                  'panda express', 'STARBIRD FOSTER CITY CA', 'HOLY GELATO',
                  'WHATABURGER', 'pokitrition', 'carls jr', 'cafe', 'in n out', 'five guys', 'doordash', 'chipotle',
                  'taco bell', 'subway', 'pita pit',
                  'sosoba', 'jersey mikes', 'cheba hut', 'cereal killerz', 'ike', 'fatty dad', 'tous les', 'atl wings',
                  '5guys', 'ocotillo',
                  'beach liquors', 'Big Papa', 'bosa donuts', 'tpumps', 'sweets 4 treats',
                  'TWO HIPPIES BEACH HOUSE PHOENIX AZ', 'KFC', 'STANFORDSTADIUMCONCESSIO', 'JACK IN THE BOX',
                  'CKE*SOM TUM STATION', r"\*HOWIE'S ARTISAN PIZZ", "Nick the Greek", "Popeyes",
                  "8CHICK-FIL-A #04448 DALY CITY CA", 'Shake Shack', 'SOMI SOMI', "SOMISOMI", "IN-N-OUT",
                  r'COW PALACE \(FOOD\)', 'COCOBERRY', r'\*Lemon SQUEEZY', r'\*patricks famous pi', 'sapore express',
                  'MEDITERRANEAN', 'ROCKNWRAPS', r'\*CARIBBEAN', r'TST\*ONE FOODSERVICE', 'THE SANDWICH',
                  r'MEDITERRAN\*', r'DINE--ONLINE\*ARARATKEB', 'FAIRMONT SAN', 'CHECKERS', 'BOX LUNCH', 'ATLAS BREW',
                  'CAVA', 'THE DETOUR', 'SOURDOUGH', r' \*VOYAGER-SAN', r'\*PINK PINK', 'WENDY\'S',
                  'BAKERY AND PASTRY'],
    'restaurant': ['olive', 'seven mile', 'crack shack', 'haus', 'charlie', 'rotten john', 'pier', 'ollie', 'mekong',
                   'valley bar', 'bop', 'breakfast',
                   'crescent', 'thai basil', 'happy', 'la parrilla', 'ramen', 'first watch', 'endgame', 'grill',
                   'postino', 'bob & edith', 'meat pies',
                   'tom douglas', 'teaspoon', 'whining', '2LEVY@BANK1 PHOENIX AZ', 'slice', 'lebanese', 'noodles',
                   'superstition meader', 'union 76',
                   'PIZZERIA BIANCO PHOENIX AZ', 'SIZZLE KOREAN BBQ PHOENIX AZ', 'OREGANOS 1028 SCOTTSDALE AZ', 'FOGO',
                   'THE THEODORE PHOENIX AZ', "RISE PIZZERIA BURLINGAME", "ICICLES - MOUNTAIN MOUNTAIN VIEWCA",
                   "URBAN RITUAL SAN MATEO", "SALT & STRAW SAN JOSE", "ROYAL DONUT BURLINGAME",
                   "OLDE TYME KETTLE K FRESNO", "GOTHAM ENTERPRISE SAN", "GELATAIO SAN CARLOS", "CHERIMOYA BURLINGAME",
                   "FRANK GRIZZLY", "ARIZMENDI BAKERY C SAN", "WAHLBURGERS BURLINGAME BURLINGAME CA", "CREPEVINE",
                   "RASPADOS Y ANTOJITAS LA", "ANCHOR PUBLIC TAPS SAN FRANCISCOCA", "ASIAN BOX BURLINGAME CA",
                   "AYTTHAYA THAI SEATTLE WA", "OLD SPAGHETTI FACTORY", "SOL SCOTTSDALE", "KJ'S BAR SEATTLE",
                   "HONG KONG BANJUM PAIK'S", "HAN SUNG BBQ", "ALHAMBRA IRISH HOUSE", "THE PLANING MILL", "HABIT",
                   "THE NEW STAND", 'SFO THE PLANT', 'HYATT REG PHOENIX F&B', 'FLIGHTS BURLINGAME', 'COCOLA',
                   'BRUNDAVAN INDIAN CUISINE', 'APPLE FRITTER', 'INMOTION-814', 'NEW ERITREA',
                   'CHUKAR CHERRY COMPANY SEATTLE WA', 'jack\'s fish', 'maryvale', 'tony', 'GUANAQUITO RESTAURANT',
                   'MANNA SAN FRANCISCOCA', 'DISH N DASH 1ST SAN JOSE CA', 'MANNA SAN FRANCISCOCA',
                   r'SQ \*SUAVECITO BIRRIA', 'MEET FRESH', 'JAMIES PLACE SAN FRANCISCOCA',
                   r'SQ \*URLBINGAME BURLINGAME CA0001152921512317149246', "SMOKING GUN", "Dive Bar & Grill",
                   "Copenhagen Bakery", "Donut Bar", "Mama Made Thai", "Hapa's Brewing", "Donut Delite", "Brewing",
                   "Wicked Popcorn", "Pho Nam", "Samos Greek Isl", "THE CRUCI WASHINGTON DC", "Love Burn Chicken",
                   "Bar", "Grill", "THE BLACKSMITH REDWOOD CITY CA", "GHIRARDELLI", "GOLDEN CORRAL #2486 LARGO MD",
                   "HOMEROOM TO GO OAKLAND CA", "CURRY LEAF SAN FRANCISCOCA", "CRACKED & BATTERED",
                   "CHARLEYS PHILLY STEAKS 9 ARLINGTON VA", "BIMBOS 365 CLUB SAN FRANCISCOCA",
                   "JOSE A MARTINEZ V CAMPBELL CA", "NEVERIA LOS MOCHIS SAN DIEGO CA",
                   "HEALTH PUB- FASHIO ARLINGTON VA", "HILL PRINCE WASHINGTON DC", "SWAMIS NORTH PARK",
                   "ANDERSEN BAKERY", "MQ HEALTHY FAST FOOD", r"STORE\*BENS FAST FOOD", r"TST\* RAD RADISH",
                   "CURRY UP NOW REDWOOD CITY", "WOOLY PIG", "WAKE CUP", "LUCHO", "THE SPORTS PAGE INC MOUNTAIN VIEW*",
                   "2OAKLAND ARENA OAKLAND CA", "THE ITALIAN HOMEMA", "LA DOLCE VITA GELATO MILPITAS CA",
                   "SHAKE SHACK SAN MATEO", "ROSE BOWL SPECIAL SAN DIEGO", "SIMPLYCAKE SF SAN MATEO", "ASACKOFPOTATOE",
                   "BEST OF THAI NOODLE SAN FRANCISCOCA", "CHE FICO SAN", "UNUSUAL TIMES EWR NEWARK",
                   "BERT\'S STADIUM SPORTS BA SUNNYVALE", "DOG PATCH SFO T1 SAN FRANCISCOCA",
                   "ROSE BOWL SPECIAL PASADENA CA", "FARMER\'S FRIDGE CHICAGO IL02677R", "DAISO", "GIO GELATO ITALIAN",
                   "SPECIAL POPUP MENLO", "THE EPICUREAN TRAD SAN", "URBAN REMEDY UNION S SAN FRANCISCOCA",
                   "TOMO SUSHI AND TERIYAKI DALY CITY CA", 'B STAR SAN', 'CASCAL RESTAURANT MOUNTAIN VIEWCA',
                   'EB GOLDEN HOUR NIGHT 801-413-7200 CAAPPLE PAY ENDING IN 1455',
                   'BOBA GUYS POTRERO SAN FRANCISCOCA0001152921512980421691',
                   'ADORABLE BAKERY LL SAN FRANCISCOCA0001152921513049179594',
                   'MONARCH BEVERAGE C SAN FRANCISCOCA0002305843018002898080', 'URBAN KITCHEN BURLINGAME',
                   'SANDYS SAN FRANCISCOCA00104288013715099959AA', 'SOMI SOMI', 'AMORINO', 'CHARLEYS PHILLY STEAKS',
                   'REGENCY BAL SAN', 'E-TEA 49', 'KUNG FU TEA', 'GRUBHUBTAQUERIALACUMB', 'IRVING SUBS - 12TH SAN',
                   'ZAREEN\'S', 'ARLINGTON DRAFTHOUSE',
                   'BY ME BOBA TEA', 'SWEET MOMENT', 'LIVING ROOM COFFE CAMPBELL', 'ICICLES - SAN MAT SAN MATEO',
                   'SAN JOSE CONVENTION', 'AMORINO SAN FRANCI', 'EDINBURGH CASTLE SAN FRANCISCOCA', 'MILLBRAE KEBABS',
                   'GRUBHUBTAQUERIALASPEN', 'SHAKE SHACK PALO ALTO', 'SABY THAI RESTAU', 'CHICK-FIL-A',
                   'RUBY THAI KITCHEN MONTGO', 'ZHANGLIANG MALATANG', 'OPEN CITY', 'DAEHO KALBIJJIM', 'THAI RESTAU',
                   'NEW ENGLAND LOBST', 'B STREET & VINE', 'MESKIE\'S ETHIOPIA RESTAU BERKELEY CA',
                   'HAMANO SUSHI SAN FRANCISCOCA', 'GIORDANO', 'Kokko', 'The Boardroom', 'OLFACTORY', 'FOLSOM',
                   'THE REFUGE', 'BURLINGAME BURLINGAME', 'EMA HOUSE', 'TAQUERIA', 'SARA LEE', 'JOE THE JUICE',
                   'PALMETTO', 'SUNRIGHT TEA', 'GAI CHICKEN', 'JEFFREYS HAMBURGER', 'ZEITGEIST', 'PINKPINK',
                   '3RD AVENUE SPORTS', 'APPLE FRIT', 'ASIAN BOX', 'OFF THE GRID', '/*KABAB & CURRYS', 'Lao Table',
                   r'\*tonkatsu', 'WAFFLE AND BERRY', 'MAISON ALYZEE', r'\*Primo\'s Gourmet', r'\*Papito Hayes',
                   'VIETNOMS', r'\*Mediterranean Kebab', r'\*Pagan Idol', 'NANDO\'S', r'DD \*COBANIGYROSKEBA',
                   r'SQ \*SAFRON', 'ENSSARO', 'JASPERS', 'MARVIN GARDENS', 'CMH COLUMBUS MKPLACE', 'BAKERY'],
    'food_delivery': ['GrubHub', 'DOORDASH', 'UBER EATS', 'POSTMATES', r'DOORDASH\*', 'The napper tandy'],
    'coffee': ['dutch bros', 'dunkin', 'starbucks', 'peet', 'caffe', 'SCOTTSDALE Q SCOTTSDALE', 'coffee',
               'MINTS & HONEY', 'COCA COLA'],

    'work_lunch': ['cavasson'],

    # Bills for transportation
    'ride_share': ['uber', 'lyft', 'razor ride', 'spin mobility', r'Lim\*Ride', r"Lim\*Sub", r'.*scooter.*', 'METRO'],
    'gas_station': ['shell', 'exxon', 'chevron', 'arco', 'circle k', 'love\'s', 'orca', 'QT', 'FUEL 24 7 WESTBOROUGH',
                    'DIMOND FOOD MART LLC', "Royal Farms", "PETRO*", "A&A GAS BURLINGAME CA", "Speedway", "Valero",
                    'A ONE GAS SAN LORENZO CA', 'FUEL', 'K On the Go', 'SHEETZ', 'WEIS'],
    'parking': ['parking', 'showcase mall', 'commutifi', 'meter', "MARINER GARAG OXON HILL MD",
                "PABC MULTI SPACE 3 BALTIMORE MD", "Parkmobile", "Parkwhiz", "UNION STREET PLAZA GARAG SAN FRANCISCOCA",
                "IMPARK00270201A SAN FRANCISCOCA00811R", 'PRESTIGE PARK LLC SAN FRANCISCOCA',
                'MONTCLAIR GARAGE PRK MBL OAKLAND CAAPPLE PAY ENDING IN 1455',
                'CITY OF BURLINGAME-PARKI BURLINGAME CA', 'PCI L761 BROADWAY PLACE WALNUT CREEK CA',
                'MONTCLAIR GARAGE'],
    'air_travel': ['american', 'alaska', 'united airlines', 'ALLIANZ TRAVEL INS',
                   "TRAVEL GUARD GROUP INC 877-934-8308 WI", "Delta", 'uscustoms'],
    'lodging': ['GREEN TORTOISE HOSTEL', 'hilton', 'hostel', 'MOTEL SAKURA GLENDALE CA'],
    'tolls': ['fastrak'],

    # Bills for applications
    'netflix': ['netflix'],
    'spotify': ['spotify'],
    'hulu': ['hulu'],
    'viki': ['viki'],
    'nba_league_pass': ['nba'],
    'hbo': ['hbo'],
    'crunchy_roll': ['crunchy'],
    'blossom': ['blossom'],
    'bumble': [r'\*BUMBLE'],
    'meal_lime': ['meal lime'],
    'apple_tv': ['apple tv'],
    'grammarly': ['grammarly'],
    'youtube_tv': ['youtube', r'GOOGLE \*YT PRIMETIME'],
    'nvidia': ['nvidia'],
    'pay_wall': ['hearst', 'sportplan'],
    'disney': ['disney'],
    'medium': ['medium'],
    'fubo_tv': ['fubotv'],
    'breaking_points': ['KRYSTAL AND SAAGAR', 'BREAKING POINTS'],
    'coursera': ['COURSRA'],
    'onepass': ['onepass'],
    'chatgpt': ['chatgpt'],
    'paramount_plus': ['paramount'],
    'obsidian': ['obsidian'],
    'adobe': ['adobe'],
    'basketball_monster': ['BASKETBALL MONSTER'],
    'Duet': ['DUET'],
    'do_the_bay': ['Dothebay'],
    'real_python': [r'PADDLE\.NET'],
    'google_colab': [r'GOOGLE \*COLAB'],
    'lickd': ['LICKD'],
    'google_one': [r'\*Google one'],
    'microsoft365': [r'MICROSOFT\*STORE'],

    # Bills for Quick purchases
    'conv_store': ['walgreens', 'cvs', '7-eleven', 'BEVERAGES & MORE', 'WMT PLUS'],
    'everything_store': ['amazon', 'walmart', 'target', 'staples', 'FIVE BELOW', 'BED BATH'],
    'airport': ['dca', 'faber news', 'vending', 'SKYLINE NEWS AND GIFTS', 'URBAN MKT CONCOURSE D SEATTLE WA',
                'HUDSONNEWS', 'TULSA AIRPORT', r'668 - \@EASE'],

    # Bills for fun
    'date_w_vincent': ['COCUSOCIAL COOKING CLA 9299303900 NY'],
    'oculus': ['oculus', 'widmo', 'gauss labs'],
    'onlyfans': ['onlyfans', 'ccbill'],
    'concerts': ['corinne', 'the van buren', 'fleet', 'phoenix performing', 'moore theater', 'ritt Momney',
                 'AKCHIN PAVILION AMPHI PHOENIX AZ', "GREAT AMERICAN MUS", 'see tickets', 'jacob collier', "MIDWAY",
                 "FILLMORE", "UNKNOWN MORTAL ORC", "JAWNYWALLICE", 'THE GREEK THEATER SAN',
                 'BENNYSINGSDANA 800-965-4827', 'SOFAR SOUNDS', 'THE ANTHEM', 'SCOOTERS WASHINGTON', r'AXS\.COM',
                 'REGENCY BALLROOM', 'WARFIELD', 'WATERSTRIDER', r'\*FOSTER THE PEOPLE\-', r'\*FOX THEATER'],
    'music': [r'MUSIC \& ARTS 74'],
    'concert_merch': ['WILLIS', 'EXPLOSIONS IN THE SKY', 'INTERVALS PORTLAND'],
    'music_festival': ['frontgate', 'festival', 'innings', 'OUTSIDELAND MERCH'],
    'general_festival': [r'\*That Garlic Stuff', 'GARLIC FETIV', 'STOCKTON'],
    'comedy': ['stand up', r'\*SHENG WANG'],
    'live_nba': ['suns', 'sacaramentokings', 'unlv web', 'QUINTEVENTS', 'golden1', 'ticketmaster', 'bill graham',
                 'AMK CAPITAL ONE ARENA CN WASHINGTON DC', "FUZE TECHNOLOGY"],
    'political_cause': ['planned parent', 'GOFUNDME HOMOSEXUAL WA REDWOOD CITY CA'],
    'zoo': ['zoo'],
    'sport_stuff': ['dicks', 'SPORTSLINE', 'BIG 5 SPORTING GOODS', 'SP BASKETBALL'],
    'movies': ['amc', 'ati', 'theatre', 'fandango', 'cine', 'cinema', "PRIME VIDEO"],
    'live_baseball': ['diamond', 'ORACLE PARK', 'OAKLAND COLISEUM'],
    'museum': ['chihuly', 'musical instrument', 'botanical', 'grand c', 'space needle', "spy museum",
               "CALIFORNIA ACAD SCIENCES SAN FRANCISCOCA", "CA ACAD. OF SCIENCES O SAN FRANCISCOCATXNREF12345",
               'SFMOMA MUSEUM STORE SAN FRANCISCOCA00690R'],
    'tennis': ['usta', 'a-z tennis', 'SAN MATEO PARKS AND RECR 650-522-7408 CA', 'SLINGERBAG 8554233729 CA',
               'PALO ALTO TENNIS SHOP PALO ALTO CA', 'TENNIS WAREHOUSE 8008836647 CA',
               'PRESIDIO COMMUNITY YMCA SAN FRANCISCOCA'],
    'home': ['casper', "LOWE\'S OF SAN BRUNO, CA SAN BRUNO CA", 'SAN MATEO SERVICE FEE 650-522-7408 CA'],
    'parks_rec': ['hoover dam', 'MUIR SPACE 4288 MILL VALLEY CA', 'MUIR WOODS'],
    'video_games': ['steam', 'EPIC GAMES', 'Square enix', 'Valve'],
    'live_event': ['showtime', "ESPN PLUS 402-935-7733 CA", "PUBLIC WORKS SAN FRANCISCOCA", 'Stubhub',
                   r'TCKTWEB\*REGALSQUALORYU', 'RADIOHEAD-KID A'],
    'vr_arcade': ['r u 4 real', "sandbox vr"],
    'self_care': ['OCEANIC FOOT SPA BURLINGAME', 'stretchlab', 'NOLIMITSCRYO', 'KUME FLOAT'],
    'electronics': ['Best Buy', 'SP DROP.COM', "KEYCHRON.COM", 'CENTRAL COMPUTERS', 'MICRO CENTER-TOWSON',
                    r'GAMERSNEXUS\.NET'],
    'bowling': ['712 GARAGE LANES'],
    'convention': ['Katsu Kei', 'Hella Kinketsu', 'SALE PRICE TAXES FORT WASHINGTMD',
                   'NEON CULTURE RIDGELAND MS0002305843017072974051', 'CARRIERCON', 'MOSCONE CENTER'],
    'kids': ['scramble', "THE CAPITAL WHEEL OXON HILL MD", 'AUTOBAHN', 'KRAKEN KOURTS',
             '0795 HOLIDAY PHOTO BETHESDA MD'],
    'swimming': ['YMCA OF SAN FRANCISCO SAN FRANCISCOCA'],
    'manga': ['MONSTERS WE MAKE'],

    # Bills for personal items (clothing etc)
    'clothing': ['famous footwear', 'foot locker', 'ross', 'straw and wool', 'rei', 'unlv tmc', 'h&m', 'simply seattle',
                 r'gdp\*vimraj', 'bonobos', 'uniqlo', 'mens wearhouse', 'foot', 'runner',
                 "GOODWILL OF SAN FRANCI SAN FRANCISCOCA", "MW TUX", 'PARK LIFE SAN', 'KORI WORLD', 'VOguishli',
                 'SPIRIT HALLOWEEN', 'Nordstrom', 'Phantogram Shop', 'FEVOGM', 'RDMCLOTHINGART', 'UNLIMITED POTENTIAL',
                 'POSHMARK', 'TAGWEARSTORE', 'SP WAYOFWADE', 'JOANN STORES', 'HOT TOPIC'],
    'wallet': ['minter goods'],
    'hair_supplies': ['waba hair', 'EBONY BEAUTY'],
    'home_supplies': ['home depot', 'OFFICE DEPOT', 'MAIDO STATIONERY', 'MINISO DEPOT MONTGOMERY BETHESDA MD',
                      'GROUPGREETING', 'Kohler'],
    'desk_supplies': ['TOPDRAWER', 'HOBBY-LOBBY'],
    'moving': ['u-haul', 'pod', 'Michaels', 'taskrabbit', 'extra space'],
    'mailing': ['FEDEX', 'USPS', "UPS"],
    'fitness': ['Sports Nutri', 'YOGASIX', "OTF", "02738 PF SAN BRUNO CA SAN BRUNO CA", "Rumble", "F45", "CLUBPILATE*",
                "SP MANDUKA 3104261495 CAAPPLE PAY ENDING IN 1455", "ONNIT", 'MOVEMENT SANTA CLARA 5413165747 CO',
                'classpass', '24 Hour Fitness', r'\*6 Pack', r'\*WELLTECH APPS', 'GNC'],
    'motor_gear': ['GP SPORTS - CA SAN JOSE CA'],
    'jewelry': ['CLAIRE', 'Etsy*', "Rose Gold*"],
    'sports_gear': ['SPORTS COLLECTIBLES', 'TEAMFANSHOP 855-210-8831 FL', "NH ENTERTAINMENT L OXON HILL MD"],

    # Bills for learning
    'learning': ['datacamp', 'interactive', 'udemy', 'linkedin', 'transcript', r'\*play', 'book', 'GA TECH', 'WILEY',
                 'COURSE HERO', 'LEETCODE', "LEANPUB 6049168017 CAN", "GA INST TECH",
                 'QUIZLET.COM 510-495-6550 CAP-08481062', 'TYPESET', 'strata scratch', r'CHENG \& TSUI',
                 r'\.natesilver\.', 'Coursera.ORG', 'STATQUEST'],
    'professional': ['linkedin', 'dataspell', 'Finance Plus', 'HARVARD BUS', 'wework', 'github',
                     "INTERVIEW QUERY 6504513704 CA", "FANTASYMATH.COM 9204509068 WI", 'APPY PIE 7039964429 VA',
                     'DROPDECK 7372426703 MD', 'CAS PROF ONLINE'],
    'motor_cycle_license': ['PACIFIC MOTORCYCLE TRAIN LIVERMORE CA', 'CYCLE GEAR', 'SP YN MOTO'],

    # Bills for medical services
    'medical': ['banner', 'BHSM REHABILITATIO', 'CARBON HEALTH', 'SUTTR HLTH', r'MED\*MERITUS', r'MED\*KASRA'],
    'dental': ['river run', "Dental"],
    'vision': ['H. Chiem O.D.', 'CHIEM', 'PAMF PA ONLINE BILL PAY 877-252-1777 CA', 'OAKLEY B'],
    'physical_therapy': ['Ayurbliss Phys', 'Ayurbliss LLC', 'ORTHOPEDIC SPORTS'],

    # Bills for China
    'GT_China': ['ALIPAY BEIJING CHN1442.00 @ 00000000.1385714 CNY', 'ALIPAY BEIJING CHN2163.00 @ 00000000.1383033 CNY',
                 'MACYS HILLSDALE SAN MATEO CA', 'Consulate', r'Google \*fi', r'PAYPAL \*C P 888-221-1161 CA',
                 'HAAM DIM HONG KONG HKG70.00 @ 00000000.1300000 HKD'],
    'Hawaii': ['Blue Note', ' Hawaii', r'\*Honolulu', 'WAIKIKI', 'HONOLULU', 'MOANA', 'HI'],

    # Bills for Friends
    'Kaiya_Birthday': ['WEST COAST CONFECTION 4252207805 CA', r"CRUMBL\* SHIPPING 8014101313 UT"],
    'Julie_Birthday': ['PAW PATCH PASTRIES DALY CITY'],
    'Jayelin': ['SANDBOXX'],
    'credits': ['AUTOMATIC STATEMENT CREDIT', 'A         S         C', 'A S C'],
    '1946': ['1946 THE MOVIE'],
    'black_in_tech': ['EB BLACK TECH GATHERI'],
    'Tahjei': [r'\*Peak Living Property'],

    'misc': ['EPOCH.COM GUERNSEY GGY', 'GOFUNDME', 'TJMAXX', 'CASH APP LUCIAN', r'\*CROWN CREATED', 'FH MILPITAS',
             'RPS UNIVERSAL', 'LIFETIME TENNIS', 'CHM SUPPLY', r'\*KWMR OLEMA']
}
for k, tokens in mapping.items():
    mask = False
    for t in tokens:
        m = ytd_data['Description'].str.contains(t, case=False, regex=False, na=False)
        mask = m if mask is False else (mask | m)
    ytd_data.loc[mask, 'Short_Desc'] = k

for k, v in mapping.items():
    ytd_data.loc[ytd_data.Description.str.contains('|'.join(v), case=False), 'Short_Desc'] = k
    ytd_data.loc[
        (ytd_data.Description.str.contains('apple', case=False)) & (ytd_data.Amount == 15.89), 'Short_Desc'] = 'hbo'
    ytd_data.loc[
        (ytd_data.Description.str.contains('apple', case=False)) & (ytd_data.Amount == 16.95), 'Short_Desc'] = 'hbo'
    ytd_data.loc[
        (ytd_data.Description.str.contains('apple', case=False)) & (ytd_data.Amount == 20.12), 'Short_Desc'] = 'hbo'
    ytd_data.loc[
        (ytd_data.Description.str.contains('apple', case=False)) & (ytd_data.Amount == 6.35), 'Short_Desc'] = 'shiloh'
    ytd_data.loc[
        (ytd_data.Description.str.contains('apple', case=False)) & (ytd_data.Amount == 13.21), 'Short_Desc'] = 'shiloh'
    ytd_data.loc[
        (ytd_data.Description.str.contains('apple', case=False)) & (ytd_data.Amount == 10.59), 'Short_Desc'] = 'netflix'
    ytd_data.loc[
        (ytd_data.Description.str.contains('apple', case=False)) & (ytd_data.Amount == 5.29), 'Short_Desc'] = 'apple_tv'
    ytd_data.loc[
        (ytd_data.Description.str.contains('apple', case=False)) & (ytd_data.Amount == 7.41), 'Short_Desc'] = 'apple_tv'
    ytd_data.loc[
        (ytd_data.Description.str.contains('apple', case=False)) & (ytd_data.Amount == 38.14), 'Short_Desc'] = 'babbel'
    ytd_data.loc[(ytd_data.Description.str.contains('apple', case=False)) & (
            ytd_data.Amount == 14.99), 'Short_Desc'] = 'nba_league_pass'
    ytd_data.loc[(ytd_data.Description.str.contains('apple', case=False)) & (
            ytd_data.Amount == 3.17), 'Short_Desc'] = 'meal_lime'
    ytd_data.loc[
        (ytd_data.Description.str.contains('apple', case=False)) & (ytd_data.Amount == 31.79), 'Short_Desc'] = 'blossom'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 553.89), 'Short_Desc'] = 'iphone_14'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 53.72), 'Short_Desc'] = 'iphone_case'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 76.31), 'Short_Desc'] = 'the_athletic'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 37.09), 'Short_Desc'] = 'yahoo_fantasy_plus'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 9.99), 'Short_Desc'] = 'apple_care'
    ytd_data.loc[ytd_data.Description.str.contains(r"APPLE\.COM", case=False) & (
            ytd_data['Amount'] == 9.99), 'Short_Desc'] = 'apple_care'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (ytd_data['Amount'] == 26.49) & (
            ytd_data.Date == '01/2022'), 'Short_Desc'] = 'anki_mobile'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (ytd_data['Amount'] == 26.49) & (
            ytd_data.Date == '11/2022'), 'Short_Desc'] = 'bumble'
    ytd_data.loc[
        (ytd_data.Description.str.contains('venmo', case=False)) & (ytd_data.Amount == 74.68), 'Short_Desc'] = 'adity'
    ytd_data.loc[(ytd_data.Description.str.contains('venmo', case=False)) & (
            ytd_data.Amount == 88.39), 'Short_Desc'] = 'nick&julie'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 63.59), 'Short_Desc'] = 'grocery_ai'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 42.92), 'Short_Desc'] = 'audiobooks'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 35.77), 'Short_Desc'] = 'audiobooks'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 31.75), 'Short_Desc'] = 'audiobooks'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 24.79), 'Short_Desc'] = 'audiobooks'
    ytd_data.loc[ytd_data.Description.str.contains("apple", case=False) & (
            ytd_data['Amount'] == 105.99), 'Short_Desc'] = 'audiobooks'


In [10]:
ytd_data[ytd_data['Short_Desc'].isnull()].sort_values(by='Amount', ascending=False)

,index,Trans. Date,Post Date,Description,Amount,Category,Sort_Date,Date,Account,Short_Desc


In [11]:
reocurring_list = ['breaking_points', 'spotify', 'netflix', 'hulu', 'viki', 'nba_league_pass', 'hbo', 'crunchy_roll',
                   'blossom', 'meal_lime', 'apple_tv', 'grammarly', 'youtube_tv', 'nvidia', 'pay_wall', 'disney',
                   'medium', 'fubo_tv', 'nvidia', 'stretch_zone', 'political_cause', 'state_farm', 'cell_phone', 'gym',
                   'stretch_zone', 'youtube', 'coursera', 'the_athletic', "onepass", "chatgpt", "paramount_plus",
                   'babbel', 'obsidian', 'physical_therapy', 'apple_care', 'Duet', 'adobe', 'do_the_bay', 'real_python',
                   'google_colab', 'lickd', 'microsoft365', 'google_one']
ytd_data.loc[ytd_data.Short_Desc.isin(reocurring_list), 'Reoccurring_Flag'] = 'Yes'
ytd_data.loc[~ytd_data.Short_Desc.isin(reocurring_list), 'Reoccurring_Flag'] = 'No'

mapping = {
    'Housing': ['home', 'home_supplies', 'lodging'],
    'Transportation': ['ride_share', 'gas_station', 'parking', 'bus_fare', 'tolls',
                       "motor_cycle_license"],
    'Food': ['fast_food', 'restaurant', 'groceries', 'coffee', 'work_lunch', 'food_delivery'],
    'Insurance': ['state_farm'],
    'Utilities': ['cell_phone', 'aps', 'car', 'storage'],
    'Medical': ['medical', 'dental', "vision", "physical_therapy"],
    'Government': ['taxes', 'fine'],
    'Savings': [],
    'Personal_Spending': ['anki_mobile', 'conv_store', 'clothing', 'coursera', 'learning', 'staples', 'target',
                          'stretch_zone', 'bookstore', 'professional', 'political_cause', 'gym', "onepass",
                          'barbershop', 'wallet', 'everything_store', 'walmart', 'clothes', 'hair_supplies',
                          'bed_bath_and_beyond', 'amazon', 'five_below', 'big_5_sporting_goods', 'leet_code',
                          'sports_line', 'iphone_14', 'iphone_case', 'apple_care', "sports_gear", "chatgpt", "fitness",
                          "jewelry", "motor_gear", 'obsidian', 'self_care', 'adobe', 'Duet', 'do_the_bay',
                          'real_python', 'google_colab', 'lickd', 'desk_supplies', 'concert_merch', 'grocery_ai',
                          'vpn_router', 'conferences', 'microsoft365', 'google_one'],
    'Recreation_Entertainment': ['babbel', 'breaking_points', 'spotify', 'netflix', 'hulu', 'viki', 'nba_league_pass',
                                 'hbo', 'crunchy_roll', 'blossom', 'meal_lime', 'apple_tv', 'grammarly', 'youtube_tv',
                                 'nvidia', 'pay_wall', 'disney', 'medium', 'fubo_tv', 'onlyfans', 'oculus', 'concert',
                                 'movies', 'museum', 'music_festival', 'live_nba', 'nvidia', 'comedy', 'live_baseball',
                                 'concerts', 'parks_rec', 'tennis', 'vr_arcade', 'video_games', 'live_event',
                                 'sport_stuff', 'zoo', 'electronics', 'bowling', 'youtube', 'the_athletic',
                                 'yahoo_fantasy_plus', 'bumble', "paramount_plus", "convention", 'Pre_Europe_Trip',
                                 "Europe_Trip", 'basketball_monster', 'black_in_tech', 'audiobooks',
                                 'general_festival', 'swimming', 'music','manga'],
    'Travel': ['Hawaii', 'GT_China', 'air_travel', 'airport'],
    'Misc': ['unsure', 'vending', 'moving', 'mailing', 'dump', 'misc'],
    'People': ['tahjei', 'don', 'dad', 'jayelin', "shiloh", "kids", 'Kaiya_Birthday', 'adity', 'nick&julie',
               'Julie_Birthday', 'mom', '1946', 'date_w_vincent', 'Tahjei', 'Jayelin'],
    'Payment_and_Interest': ['payment', 'interest', 'cashback_bonus', 'credits', 'fees']
}

for k, v in mapping.items():
    ytd_data.loc[ytd_data.Short_Desc.isin(v), 'Category_2'] = k

In [12]:
ytd_data[ytd_data['Category_2'].isnull()].sort_values(by='Description', ascending=False).sort_values(by='Short_Desc',
                                                                                                     ascending=False)

,index,Trans. Date,Post Date,Description,Amount,Category,Sort_Date,Date,Account,Short_Desc,Reoccurring_Flag,Category_2


In [13]:
x = ytd_data.groupby(['Reoccurring_Flag']).size().reset_index(name='count')

x.sort_values(by='count', ascending=False)

,Reoccurring_Flag,count
0,No,809
1,Yes,118


In [14]:
ytd_sorted = ytd_data.sort_values(by='Sort_Date')

# Define the new row with datetime objects for date columns
new_row = pd.DataFrame({
    'index': [0],  # Ensure all entries are in list format to align with DataFrame structure
    'Trans. Date': [datetime.strptime('1/1/2023', '%m/%d/%Y')],  # Matching format and conversion
    'Post Date': [datetime.strptime('1/1/2023', '%m/%d/%Y')],
    'Description': ['Balance'],
    'Amount': [227.70],
    'Category': ['Balance'],
    'Short_Desc': ['Balance'],
    'Reoccurring_Flag': ['No'],
    'Category_2': ['Balance'],
    'Date': [pd.to_datetime('1/1/2023')],  # This is a string representation of the date, so it's okay
    'Year': [2023.0],
    'Sort_Date': [datetime.fromisoformat('2023-01-01')],  # Use datetime to create a datetime object
    'Account': ['discover']
}, index=[0])

# Concatenate the new_row DataFrame with the existing ytd_sorted DataFrame
ytd_sorted = pd.concat([new_row, ytd_sorted], ignore_index=True)

# Ensure 'Sort_Date' is also in datetime format for the entire DataFrame
ytd_sorted['Sort_Date'] = pd.to_datetime(ytd_sorted['Sort_Date'])
ytd_sorted

,index,Trans. Date,Post Date,Description,Amount,Category,Short_Desc,Reoccurring_Flag,Category_2,Date,Year,Sort_Date,Account
0,0,2023-01-01,2023-01-01,Balance,227.70,Balance,Balance,No,Balance,2023-01-01,2023.0,2023-01-01,discover
1,0,2024-01-01,2024-01-01,CHATGPT SUBSCRIPTION 4158799686 CA,20.00,Merchandise,chatgpt,Yes,Personal_Spending,2024-01-01,NaN,2024-01-01,discover
2,1,2024-01-01,2024-01-01,JFF PUBLICATIONS WILTON MANORSFL,24.99,Merchandise,movies,No,Recreation_Entertainment,2024-01-01,NaN,2024-01-01,discover
3,2,2024-01-01,2024-01-03,PLANNED PARENTHOOD ACTIO NEW YORK NY,5.00,Services,political_cause,Yes,Personal_Spending,2024-01-01,NaN,2024-01-01,discover
4,3,2024-01-01,2024-01-03,SPOTIFY P286B95411 8777781161 NY,5.99,Merchandise,spotify,Yes,Recreation_Entertainment,2024-01-01,NaN,2024-01-01,discover
...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,922,2024-12-27,2024-12-27,USPS PO 0510350110 BURLINGAME CA,44.65,Government Services,mailing,No,Misc,2024-12-01,NaN,2024-12-27,discover
924,923,2024-12-28,2024-12-28,FASTRAK CSC 415-486-8655 CAITEM TRANSFERRED FR...,50.00,Merchandise,tolls,No,Transportation,2024-12-01,NaN,2024-12-28,discover
925,924,2024-12-28,2024-12-28,ONLYFANS.COM*A 8886880458 FLITEM TRANSFERRED F...,5.00,Merchandise,onlyfans,No,Recreation_Entertainment,2024-12-01,NaN,2024-12-28,discover
926,925,2024-12-29,2024-12-29,SAFEWAY #1547 BURLINGAME CA,63.28,Supermarkets,groceries,No,Food,2024-12-01,NaN,2024-12-29,discover


In [15]:
ytd_sorted['CumAmount'] = ytd_sorted['Amount'].cumsum()
ytd_sorted['Year'] = ytd_sorted['Sort_Date'].dt.year
del ytd_sorted['index']
ytd_sorted

,Trans. Date,Post Date,Description,Amount,Category,Short_Desc,Reoccurring_Flag,Category_2,Date,Year,Sort_Date,Account,CumAmount
0,2023-01-01,2023-01-01,Balance,227.70,Balance,Balance,No,Balance,2023-01-01,2023,2023-01-01,discover,227.70
1,2024-01-01,2024-01-01,CHATGPT SUBSCRIPTION 4158799686 CA,20.00,Merchandise,chatgpt,Yes,Personal_Spending,2024-01-01,2024,2024-01-01,discover,247.70
2,2024-01-01,2024-01-01,JFF PUBLICATIONS WILTON MANORSFL,24.99,Merchandise,movies,No,Recreation_Entertainment,2024-01-01,2024,2024-01-01,discover,272.69
3,2024-01-01,2024-01-03,PLANNED PARENTHOOD ACTIO NEW YORK NY,5.00,Services,political_cause,Yes,Personal_Spending,2024-01-01,2024,2024-01-01,discover,277.69
4,2024-01-01,2024-01-03,SPOTIFY P286B95411 8777781161 NY,5.99,Merchandise,spotify,Yes,Recreation_Entertainment,2024-01-01,2024,2024-01-01,discover,283.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,2024-12-27,2024-12-27,USPS PO 0510350110 BURLINGAME CA,44.65,Government Services,mailing,No,Misc,2024-12-01,2024,2024-12-27,discover,748.33
924,2024-12-28,2024-12-28,FASTRAK CSC 415-486-8655 CAITEM TRANSFERRED FR...,50.00,Merchandise,tolls,No,Transportation,2024-12-01,2024,2024-12-28,discover,798.33
925,2024-12-28,2024-12-28,ONLYFANS.COM*A 8886880458 FLITEM TRANSFERRED F...,5.00,Merchandise,onlyfans,No,Recreation_Entertainment,2024-12-01,2024,2024-12-28,discover,803.33
926,2024-12-29,2024-12-29,SAFEWAY #1547 BURLINGAME CA,63.28,Supermarkets,groceries,No,Food,2024-12-01,2024,2024-12-29,discover,866.61


In [16]:
ytd_sorted['Counter'] = ytd_sorted.groupby('Short_Desc').cumcount() + 1
ytd_sorted

,Trans. Date,Post Date,Description,Amount,Category,Short_Desc,Reoccurring_Flag,Category_2,Date,Year,Sort_Date,Account,CumAmount,Counter
0,2023-01-01,2023-01-01,Balance,227.70,Balance,Balance,No,Balance,2023-01-01,2023,2023-01-01,discover,227.70,1
1,2024-01-01,2024-01-01,CHATGPT SUBSCRIPTION 4158799686 CA,20.00,Merchandise,chatgpt,Yes,Personal_Spending,2024-01-01,2024,2024-01-01,discover,247.70,1
2,2024-01-01,2024-01-01,JFF PUBLICATIONS WILTON MANORSFL,24.99,Merchandise,movies,No,Recreation_Entertainment,2024-01-01,2024,2024-01-01,discover,272.69,1
3,2024-01-01,2024-01-03,PLANNED PARENTHOOD ACTIO NEW YORK NY,5.00,Services,political_cause,Yes,Personal_Spending,2024-01-01,2024,2024-01-01,discover,277.69,1
4,2024-01-01,2024-01-03,SPOTIFY P286B95411 8777781161 NY,5.99,Merchandise,spotify,Yes,Recreation_Entertainment,2024-01-01,2024,2024-01-01,discover,283.68,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,2024-12-27,2024-12-27,USPS PO 0510350110 BURLINGAME CA,44.65,Government Services,mailing,No,Misc,2024-12-01,2024,2024-12-27,discover,748.33,5
924,2024-12-28,2024-12-28,FASTRAK CSC 415-486-8655 CAITEM TRANSFERRED FR...,50.00,Merchandise,tolls,No,Transportation,2024-12-01,2024,2024-12-28,discover,798.33,15
925,2024-12-28,2024-12-28,ONLYFANS.COM*A 8886880458 FLITEM TRANSFERRED F...,5.00,Merchandise,onlyfans,No,Recreation_Entertainment,2024-12-01,2024,2024-12-28,discover,803.33,99
926,2024-12-29,2024-12-29,SAFEWAY #1547 BURLINGAME CA,63.28,Supermarkets,groceries,No,Food,2024-12-01,2024,2024-12-29,discover,866.61,47


In [17]:
columns = ['Trans. Date', 'Description', 'Amount', 'Category', 'Short_Desc', 'Reoccurring_Flag',
           'Category_2', 'Date', 'Year', 'Sort_Date', 'CumAmount', 'Counter', 'Account']
discover = ytd_sorted[columns]

In [19]:
discover.to_excel('D:\\Sean\\Documents\\Personal\\Budget\\2024_output\\Curated_Bills.xlsx')